In [10]:
import numpy as np
import pandas as pd

In [11]:
import sqlite3

In [3]:
conn = sqlite3.connect('indicators_db.sqlite3')

In [4]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM sqlite_master', conn)

In [5]:
indicators

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...
2,view,v_org_indicators_all,v_org_indicators_all,0,CREATE VIEW v_org_indicators_all AS\r\nselect ...


In [6]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)

In [7]:
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


## Задание 1
Для каждой организации из main.csv  
подтяните выручку и число сотрудников за 2020, 2021 и 2022.  
Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их.  
Посчитайте дельту по выручке и сотрудникам между 2022 и 2020.  
Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID.  
Названия колонок сделайте на русском языке.

In [20]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


In [21]:
indicators = indicators[(indicators.INDICATOR_NAME == 'Выручка') | (indicators.INDICATOR_NAME == 'Число сотрудников')]

In [22]:
indicators = indicators.pivot_table('VALUE', ['ID'], ['INDICATOR_NAME', 'YEAR'])

In [26]:
indicators.columns = [col[0] + ' ' + str(col[1]) for col  in indicators.columns.values]

In [27]:
indicators.head()

,Выручка 2020,Выручка 2021,Выручка 2022,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022
ID,,,,,,
19630,31222228.0,21721921.0,NaN,NaN,15.0,9.0
19643,35670701.0,39125612.0,NaN,17.0,NaN,NaN
19647,34926614.0,23176004.0,27925131.0,16.0,15.0,NaN
19657,23410468.0,28065717.0,36074059.0,NaN,8.0,12.0
19677,NaN,30803884.0,25303592.0,17.0,9.0,NaN


In [29]:
indicators = indicators.dropna(subset=['Выручка 2020', 'Выручка 2022', 'Число сотрудников 2020', 'Число сотрудников 2022'])

In [33]:
indicators.loc[:, 'Дельта по выручке'] = indicators['Выручка 2022'] - indicators['Выручка 2020']
indicators.loc[:, 'Дельта по сотрудникам'] = indicators['Число сотрудников 2022'] - indicators['Число сотрудников 2020']

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


подтяните выручку и число сотрудников за 2020, 2021 и 2022.  
Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их.  
Посчитайте дельту по выручке и сотрудникам между 2022 и 2020.  
Для каждой организации из main.csv  
Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID.  
Названия колонок сделайте на русском языке.

In [ ]:
indicators = indicators.sort_values(['Дельта по выручке', 'ID'], ascending=[False, True])